In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris/Iris.csv
/kaggle/input/iris/database.sqlite


In [2]:
df = pd.read_csv('../input/iris/Iris.csv', index_col='Id')
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,Iris-virginica
147,6.3,2.5,5.0,1.9,Iris-virginica
148,6.5,3.0,5.2,2.0,Iris-virginica


In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Species'] = label_encoder.fit_transform(df['Species'])
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,0
2,4.9,3.0,1.4,0.2,0
3,4.7,3.2,1.3,0.2,0
4,4.6,3.1,1.5,0.2,0
5,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,2
147,6.3,2.5,5.0,1.9,2
148,6.5,3.0,5.2,2.0,2


# import Keras

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
tf.__version__

'2.2.0'

# split data into train and val

In [6]:
df_val = df.sample(frac=0.2, random_state=33)
df_val

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
97,5.7,2.9,4.2,1.3,1
66,6.7,3.1,4.4,1.4,1
30,4.7,3.2,1.6,0.2,0
55,6.5,2.8,4.6,1.5,1
135,6.1,2.6,5.6,1.4,2
101,6.3,3.3,6.0,2.5,2
25,4.8,3.4,1.9,0.2,0
18,5.1,3.5,1.4,0.3,0
138,6.4,3.1,5.5,1.8,2


In [7]:
df_val.shape

(30, 5)

In [8]:
df_train = df.drop(df_val.index)
df_train

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,0
2,4.9,3.0,1.4,0.2,0
3,4.7,3.2,1.3,0.2,0
4,4.6,3.1,1.5,0.2,0
5,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
143,5.8,2.7,5.1,1.9,2
145,6.7,3.3,5.7,2.5,2
146,6.7,3.0,5.2,2.3,2


# convert dataframe to tensor data 

In [9]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, target, shuffle=True, batch_size=10):
  dataframe = dataframe.copy()
  labels = dataframe.pop(target)
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [10]:
train_ds = df_to_dataset(dataframe=df_train, target='Species')
val_ds = df_to_dataset(dataframe=df_val, target='Species')

In [11]:
for b in train_ds.take(1):
    print(b)

({'SepalLengthCm': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([5.1, 5. , 6.6, 6. , 6.3, 6.3, 5.6, 5. , 5.1, 5.4])>, 'SepalWidthCm': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([2.5, 3.4, 2.9, 3.4, 2.5, 2.7, 2.5, 3.5, 3.4, 3.4])>, 'PetalLengthCm': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([3. , 1.5, 4.6, 4.5, 4.9, 4.9, 3.9, 1.3, 1.5, 1.5])>, 'PetalWidthCm': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([1.1, 0.2, 1.3, 1.6, 1.5, 1.8, 1.1, 0.3, 0.2, 0.4])>}, <tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 0, 1, 1, 1, 2, 1, 0, 0, 0])>)


# prepare input layer 

In [12]:
from tensorflow import feature_column

feature_columns = []
# numeric cols
for col in df.columns:
  if col == 'Species':
    continue
  feature_columns.append(feature_column.numeric_column(col, dtype=tf.float16)) 
feature_columns

[NumericColumn(key='SepalLengthCm', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None),
 NumericColumn(key='SepalWidthCm', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None),
 NumericColumn(key='PetalLengthCm', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None),
 NumericColumn(key='PetalWidthCm', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None)]

# define model 

In [13]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.DenseFeatures(feature_columns))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

# compile model

1. optimizer
2. loss
3. metircs 

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', 
             metrics=['accuracy'])

# evalute before training (evalute on random wieghts) 

In [15]:
model.evaluate(val_ds)

3/3 [==============================] - 0s 2ms/step - loss: 1.2554 - accuracy: 0.3333


[1.2554441690444946, 0.3333333432674408]

# train the model using fit fucntion 

In [16]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
12/12 [==============================] - 0s 3ms/step - loss: 1.0465 - accuracy: 0.5667 - val_loss: 1.0963 - val_accuracy: 0.5000
Epoch 2/10
12/12 [==============================] - 0s 3ms/step - loss: 0.9732 - accuracy: 0.5833 - val_loss: 0.9959 - val_accuracy: 0.4000
Epoch 3/10
12/12 [==============================] - 0s 3ms/step - loss: 0.9143 - accuracy: 0.5000 - val_loss: 0.9319 - val_accuracy: 0.3667
Epoch 4/10
12/12 [==============================] - 0s 3ms/step - loss: 0.8635 - accuracy: 0.5667 - val_loss: 0.9016 - val_accuracy: 0.4667
Epoch 5/10
12/12 [==============================] - 0s 2ms/step - loss: 0.8273 - accuracy: 0.5583 - val_loss: 0.8534 - val_accuracy: 0.3667
Epoch 6/10
12/12 [==============================] - 0s 3ms/step - loss: 0.7948 - accuracy: 0.4917 - val_loss: 0.8275 - val_accuracy: 0.3667
Epoch 7/10
12/12 [==============================] - 0s 2ms/step - loss: 0.7687 - accuracy: 0.5000 - val_loss: 0.8091 - val_accuracy: 0.3667
Epoch 8/10
12/12 [==

# model summary 

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features (DenseFeature multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  160       
_________________________________________________________________
dense_1 (Dense)              multiple                  99        
Total params: 259
Trainable params: 259
Non-trainable params: 0
_________________________________________________________________


# evaluate (after training)

In [18]:
model.evaluate(val_ds)

3/3 [==============================] - 0s 2ms/step - loss: 0.7380 - accuracy: 0.3667


[0.7379866242408752, 0.36666667461395264]

# get predictions 

In [19]:
model.predict(val_ds)

array([[0.67999995, 0.08664767, 0.23335238],
       [0.19738409, 0.37753028, 0.4250856 ],
       [0.7322463 , 0.06191027, 0.20584342],
       [0.12530789, 0.44782653, 0.4268656 ],
       [0.69290096, 0.08091567, 0.22618343],
       [0.06804539, 0.5104884 , 0.42146617],
       [0.1784598 , 0.39395687, 0.4275834 ],
       [0.09758647, 0.4582888 , 0.44412473],
       [0.16145338, 0.41047674, 0.4280699 ],
       [0.739033  , 0.0572467 , 0.20372039],
       [0.06662367, 0.5016759 , 0.4317004 ],
       [0.05654326, 0.5202141 , 0.4232427 ],
       [0.13918056, 0.41662514, 0.44419423],
       [0.6609957 , 0.09228923, 0.24671507],
       [0.13874969, 0.43471867, 0.42653164],
       [0.05467715, 0.53430545, 0.4110174 ],
       [0.1373821 , 0.43131956, 0.4312984 ],
       [0.7294377 , 0.06333629, 0.20722604],
       [0.10675792, 0.45817557, 0.43506652],
       [0.62695944, 0.10439236, 0.26864815],
       [0.18629667, 0.3777536 , 0.43594974],
       [0.0871045 , 0.481522  , 0.43137357],
       [0.

# get predickions as 0, 1, 2

In [20]:
np.argmax([0.2, 0.9, 0.01])

1

In [21]:
np.argmax([0.8, 0.19, 0.01])

0

In [22]:
np.argmax([0.08, 0.19, 0.71])

2

In [23]:
predictions = np.argmax(model.predict(val_ds), axis=1)
predictions

array([1, 2, 2, 0, 1, 2, 2, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 2, 0, 0, 1, 0])

In [24]:
len(predictions)

30

# predict a new example 

In [25]:
df.keys()

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [26]:
df.columns

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [27]:
new_data = {
    'SepalLengthCm': [5.0],
    'SepalWidthCm': [1.2], 
    'PetalLengthCm': [3.5],
    'PetalWidthCm': [0.7]
}
new_data

{'SepalLengthCm': [5.0],
 'SepalWidthCm': [1.2],
 'PetalLengthCm': [3.5],
 'PetalWidthCm': [0.7]}

In [28]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in new_data.items()}
input_dict

{'SepalLengthCm': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[5.]], dtype=float32)>,
 'SepalWidthCm': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.2]], dtype=float32)>,
 'PetalLengthCm': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[3.5]], dtype=float32)>,
 'PetalWidthCm': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.7]], dtype=float32)>}

In [29]:
model.predict(input_dict)

array([[0.18582775, 0.44110748, 0.37306482]], dtype=float32)

In [30]:
np.argmax(model.predict(input_dict))

1